### BenchMark in Interaction baselines, FIM excluded

In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.baseline_CNN_CNN import GCAModel

In [2]:
hparams = {
    'scale':'demo',
    'name':'baseline-cnn-cnn',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'filter_num':400,
    'value_dim':16,
    'head_num':16,
    'epochs':5,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title'],
    'save_each_epoch':False,
    'save_step':None,
    'train_embedding':False,
}

device = torch.device(hparams['device']) if torch.cuda.is_available() else torch.device("cpu")

In [3]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [4]:
gcaModel = GCAModel(hparams, vocab=vocab).to(device)

In [6]:
import torch
import torch.nn as nn

class GCAModel(nn.Module):
    def __init__(self,hparams,vocab):
        super().__init__()

        self.cdd_size = (hparams['npratio'] + 1) if hparams['npratio'] > 0 else 1
        self.metrics = hparams['metrics']
        self.device = torch.device(hparams['device'])
        self.embedding = vocab.vectors.to(self.device)

        self.batch_size = hparams['batch_size']
        self.signal_length = hparams['title_size']
        self.his_size = hparams['his_size']

        self.dropout_p = hparams['dropout_p']

        self.filter_num = hparams['filter_num']
        self.embedding_dim = hparams['embedding_dim']
       
        # elements in the slice along dim will sum up to 1 
        self.softmax = nn.Softmax(dim=-1)
        self.gumbel_softmax = nn.functional.gumbel_softmax
        self.ReLU = nn.ReLU()
        self.DropOut = nn.Dropout(p=hparams['dropout_p'])
        
        self.CNN = nn.Conv1d(in_channels=self.embedding_dim,out_channels=self.filter_num,kernel_size=3,padding=1)
        self.SeqCNN = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3,3), stride=(3,3)),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3,3), stride=(3,3))
        )
        
        # 64 is derived from SeqCNN
        self.learningToRank = nn.Linear(144, 1)
        # self.learningToRank = nn.Linear(self.repr_dim * self.his_size, 1)

    def _scaled_dp_attention(self,query,key,value):
        """ calculate scaled attended output of values
        
        Args:
            query: tensor of [*, query_num, key_dim]
            key: tensor of [batch_size, *, key_num, key_dim]
            value: tensor of [batch_size, *, key_num, value_dim]
        
        Returns:
            attn_output: tensor of [batch_size, *, query_num, value_dim]
        """

        # make sure dimension matches
        assert query.shape[-1] == key.shape[-1]
        key = key.transpose(-2,-1)

        attn_weights = torch.matmul(query,key)/torch.sqrt(torch.tensor([self.embedding_dim],dtype=torch.float,device=self.device))
        attn_weights = self.softmax(attn_weights)
        
        attn_output = torch.matmul(attn_weights,value)
        return attn_output

    def _news_encoder(self,news_batch):
        """ encode batch of news with 1d-CNN
        
        Args:
            news_batch: tensor of [batch_size, *]
        
        Returns:
            news_emebdding: tensor of [batch_size, *, filter_num] 
        """

        news_embedding = self.embedding[news_batch].transpose(-2,-1).view(-1,self.embedding_dim,news_batch.shape[-1])
        news_embedding = self.CNN(news_embedding).transpose(-2,-1).view(news_batch.shape + (self.filter_num,))
        news_embedding = self.ReLU(news_embedding)

        if self.dropout_p > 0:
            news_embedding = self.DropOut(news_embedding)

        return news_embedding
        
    def _fusion(self, cdd_news_embedding, his_news_embedding):
        """ concatenate candidate news title and history news title
        
        Args:
            cdd_news_embedding: tensor of [batch_size, cdd_size, signal_length, filter_num] 
            his_news_embedding: tensor of [batch_size, his_size, signal_length, filter_num] 

        Returns:
            fusion_news: tensor of [batch_size, cdd_size, his_size, signal_length, signal_length]
        """

        fusion_matrices = torch.matmul(cdd_news_embedding.unsqueeze(dim=2), his_news_embedding.unsqueeze(dim=1).transpose(-2,-1)).view(self.batch_size * self.cdd_size * self.his_size, 1, self.signal_length, self.signal_length)
        fusion_vectors = self.SeqCNN(fusion_matrices).view(self.batch_size, self.cdd_size, self.his_size, -1)     
        fusion_vectors = torch.mean(fusion_vectors, dim=-2)
        print(fusion_vectors.shape)
        return fusion_vectors
    
    def _click_predictor(self,fusion_repr):
        """ calculate batch of click probability              
        Args:
            fusion_repr: tensor of [batch_size, cdd_size, repr_dim]
        
        Returns:
            score: tensor of [batch_size, cdd_size]
        """
        score = self.learningToRank(fusion_repr)

        if self.cdd_size > 1:
            score = nn.functional.log_softmax(score,dim=1)
        else:
            score = torch.sigmoid(score)
        
        return score.squeeze()

    def forward(self,x):
        cdd_news_embedding = self._news_encoder(x['candidate_title'].long().to(self.device))
        his_news_embedding = self._news_encoder(x['clicked_title'].long().to(self.device))

        fusion_vectors = self._fusion(cdd_news_embedding, his_news_embedding)   
        score_batch = self._click_predictor(fusion_vectors)
        return score_batch
gcaModel = GCAModel(hparams, vocab=vocab).to(device)

In [7]:
record = next(iter(loader_train))

In [8]:
gcaModel(record)

torch.Size([10, 5, 144])


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [5]:
train(gcaModel, hparams, loader_train, loader_test, loader_validate, save=False)

training...
0it [00:00, ?it/s]


RuntimeError: mat1 dim 1 must match mat2 dim 0